In [23]:
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
import matplotlib.pyplot as plt
%matplotlib inline
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [24]:
from tensorflow.keras import datasets
from tensorflow.keras import layers

# (train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()
(train_x, train_y), (test_x, test_y) = datasets.fashion_mnist.load_data()

input_shape = (28, 28, 1)
num_classes = 10

In [25]:
def MyModel(input_shape_inner):
    inputs = layers.Input(shape=input_shape_inner)

    net = layers.Conv2D(32, 3, 1, padding='same', activation='relu')(inputs)
    net = layers.Conv2D(32, 3, 1, padding='same', activation='relu')(net)
    net = layers.MaxPooling2D(pool_size=(2,2))(net)
    net = layers.Dropout(0.25)(net)

    net = layers.Conv2D(32, 3, 1, padding='same', activation='relu')(net)
    net = layers.Conv2D(32, 3, 1, padding='same', activation='relu')(net)
    net = layers.MaxPooling2D(pool_size=(2,2))(net)
    net = layers.Dropout(0.25)(net)

    net = layers.Flatten()(net)
    net = layers.Dense(512, activation='relu')(net)
    net = layers.Dropout(0.25)(net)
    net = layers.Dense(num_classes, activation='softmax')(net)

    return tf.keras.Model(inputs=inputs, outputs=net, name='test_model')

In [26]:
train_x_100 = train_x[:1000]
train_y_100 = train_y[:1000]
train_x_100 = train_x_100[..., tf.newaxis]
train_x_100 = train_x_100 / 255.
train_x_100.shape, train_y_100.shape, train_x.shape, train_y.shape

((1000, 28, 28, 1), (1000,), (60000, 28, 28), (60000,))

In [27]:
model = MyModel(input_shape)

In [28]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
hist = model.fit(train_x_100, train_y_100, batch_size=32, shuffle=True, epochs=5)

Epoch 1/5


2025-05-26 11:44:24.691190: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.3343 - loss: 1.8216
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6876 - loss: 0.8859
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7504 - loss: 0.7146
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7685 - loss: 0.6185
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8030 - loss: 0.5552


### Predict 함수 사용해서 정확도 계산하기

In [29]:
test_x.shape, test_y.shape

((10000, 28, 28), (10000,))

In [30]:
test_x_4d = test_x[..., tf.newaxis] / 255.
hist = model.fit(train_x_100, train_y_100, validation_data=(test_x_4d, test_y), batch_size=32, shuffle=True, epochs=5)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.8538 - loss: 0.4658 - val_accuracy: 0.7541 - val_loss: 0.7535
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.8201 - loss: 0.5164 - val_accuracy: 0.7973 - val_loss: 0.5837
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.8599 - loss: 0.3856 - val_accuracy: 0.7673 - val_loss: 0.6023
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.8871 - loss: 0.3256 - val_accuracy: 0.8215 - val_loss: 0.5325
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.9022 - loss: 0.3199 - val_accuracy: 0.8218 - val_loss: 0.5639


In [31]:
pred = model.predict(test_x_4d)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [32]:
pred.shape, pred[0]

((10000, 10),
 array([2.10686650e-07, 7.74286093e-08, 3.58375388e-07, 2.68300710e-07,
        1.28417275e-06, 5.51050238e-04, 1.28110671e-07, 1.37849245e-02,
        7.04485137e-05, 9.85591292e-01], dtype=float32))

In [33]:
result = np.argmax(pred, axis=1)
result.shape

(10000,)

In [34]:
result, test_y

(array([9, 2, 1, ..., 8, 1, 5]), array([9, 2, 1, ..., 8, 1, 5], dtype=uint8))

In [35]:
ans_count = 0
for i in range(len(test_y)):
    if result[i] == test_y[i]:
        print(f'예측값: {result[i]}, 실제값: {test_y[i]} (정답)')
        ans_count += 1
    else:
        print(f'예측값: {result[i]}, 실제값: {test_y[i]} (오답)')
print()
print(f'정확도: {ans_count/len(test_y) * 100:.2f}%')
print('error count:', len(test_y) - ans_count)
print(f'error rate: {(len(test_y) - ans_count)/len(test_y) * 100:.2f}%')

예측값: 9, 실제값: 9 (정답)
예측값: 2, 실제값: 2 (정답)
예측값: 1, 실제값: 1 (정답)
예측값: 1, 실제값: 1 (정답)
예측값: 6, 실제값: 6 (정답)
예측값: 1, 실제값: 1 (정답)
예측값: 6, 실제값: 4 (오답)
예측값: 6, 실제값: 6 (정답)
예측값: 5, 실제값: 5 (정답)
예측값: 7, 실제값: 7 (정답)
예측값: 4, 실제값: 4 (정답)
예측값: 5, 실제값: 5 (정답)
예측값: 8, 실제값: 7 (오답)
예측값: 3, 실제값: 3 (정답)
예측값: 4, 실제값: 4 (정답)
예측값: 1, 실제값: 1 (정답)
예측값: 2, 실제값: 2 (정답)
예측값: 6, 실제값: 4 (오답)
예측값: 8, 실제값: 8 (정답)
예측값: 0, 실제값: 0 (정답)
예측값: 2, 실제값: 2 (정답)
예측값: 7, 실제값: 5 (오답)
예측값: 7, 실제값: 7 (정답)
예측값: 5, 실제값: 9 (오답)
예측값: 1, 실제값: 1 (정답)
예측값: 6, 실제값: 4 (오답)
예측값: 6, 실제값: 6 (정답)
예측값: 3, 실제값: 0 (오답)
예측값: 9, 실제값: 9 (정답)
예측값: 3, 실제값: 3 (정답)
예측값: 8, 실제값: 8 (정답)
예측값: 8, 실제값: 8 (정답)
예측값: 3, 실제값: 3 (정답)
예측값: 3, 실제값: 3 (정답)
예측값: 8, 실제값: 8 (정답)
예측값: 0, 실제값: 0 (정답)
예측값: 7, 실제값: 7 (정답)
예측값: 5, 실제값: 5 (정답)
예측값: 7, 실제값: 7 (정답)
예측값: 9, 실제값: 9 (정답)
예측값: 0, 실제값: 6 (오답)
예측값: 1, 실제값: 1 (정답)
예측값: 3, 실제값: 3 (정답)
예측값: 7, 실제값: 7 (정답)
예측값: 4, 실제값: 6 (오답)
예측값: 9, 실제값: 7 (오답)
예측값: 2, 실제값: 2 (정답)
예측값: 1, 실제값: 1 (정답)
예측값: 4, 실제값: 2 (오답)
예측값: 6, 실제값: 2 (오답)


In [36]:
import matplotlib.pyplot as plt
def plot_image(image, label):
    plt.imshow(image.squeeze(), cmap='gray')
    plt.title(f'Label: {label}')
    plt.axis('off')
    plt.show()

In [37]:
import os

In [38]:
os.getcwd()

'/Users/lsh/develop/ai-study/deep-learning'

In [39]:
os.listdir('./mnist_png/testing/0')

['8808.png',
 '6602.png',
 '9519.png',
 '1623.png',
 '3020.png',
 '5337.png',
 '8388.png',
 '5684.png',
 '1390.png',
 '6400.png',
 '5109.png',
 '6428.png',
 '5653.png',
 '2060.png',
 '5069.png',
 '9647.png',
 '6979.png',
 '8798.png',
 '4834.png',
 '1794.png',
 '5280.png',
 '8029.png',
 '5519.png',
 '8983.png',
 '5257.png',
 '7440.png',
 '1757.png',
 '3140.png',
 '5531.png',
 '5243.png',
 '8772.png',
 '215.png',
 '1959.png',
 '567.png',
 '6978.png',
 '2713.png',
 '8558.png',
 '4404.png',
 '2061.png',
 '9134.png',
 '2707.png',
 '5929.png',
 '981.png',
 '7723.png',
 '5652.png',
 '6429.png',
 '7737.png',
 '5108.png',
 '1408.png',
 '188.png',
 '6398.png',
 '7086.png',
 '5685.png',
 '611.png',
 '8389.png',
 '5450.png',
 '407.png',
 '3790.png',
 '6818.png',
 '9281.png',
 '9297.png',
 '1807.png',
 '1191.png',
 '4957.png',
 '7279.png',
 '5452.png',
 '2329.png',
 '3989.png',
 '5320.png',
 '7537.png',
 '9095.png',
 '7084.png',
 '2842.png',
 '3584.png',
 '9724.png',
 '997.png',
 '9687.png',
 '7866

In [40]:
paths = tf.io.matching_files('./mnist_png/testing/0/*.png')
paths

<tf.Tensor: shape=(980,), dtype=string, numpy=
array([b'./mnist_png/testing/0/10.png', b'./mnist_png/testing/0/1001.png',
       b'./mnist_png/testing/0/1009.png',
       b'./mnist_png/testing/0/101.png',
       b'./mnist_png/testing/0/1034.png',
       b'./mnist_png/testing/0/1047.png',
       b'./mnist_png/testing/0/1061.png',
       b'./mnist_png/testing/0/1084.png',
       b'./mnist_png/testing/0/1094.png',
       b'./mnist_png/testing/0/1121.png',
       b'./mnist_png/testing/0/1148.png',
       b'./mnist_png/testing/0/1154.png',
       b'./mnist_png/testing/0/1176.png',
       b'./mnist_png/testing/0/1188.png',
       b'./mnist_png/testing/0/1191.png',
       b'./mnist_png/testing/0/1195.png',
       b'./mnist_png/testing/0/1197.png',
       b'./mnist_png/testing/0/1203.png',
       b'./mnist_png/testing/0/1218.png',
       b'./mnist_png/testing/0/1220.png',
       b'./mnist_png/testing/0/1223.png',
       b'./mnist_png/testing/0/1231.png',
       b'./mnist_png/testing/0/1246.png

In [41]:
paths.numpy()

array([b'./mnist_png/testing/0/10.png', b'./mnist_png/testing/0/1001.png',
       b'./mnist_png/testing/0/1009.png',
       b'./mnist_png/testing/0/101.png',
       b'./mnist_png/testing/0/1034.png',
       b'./mnist_png/testing/0/1047.png',
       b'./mnist_png/testing/0/1061.png',
       b'./mnist_png/testing/0/1084.png',
       b'./mnist_png/testing/0/1094.png',
       b'./mnist_png/testing/0/1121.png',
       b'./mnist_png/testing/0/1148.png',
       b'./mnist_png/testing/0/1154.png',
       b'./mnist_png/testing/0/1176.png',
       b'./mnist_png/testing/0/1188.png',
       b'./mnist_png/testing/0/1191.png',
       b'./mnist_png/testing/0/1195.png',
       b'./mnist_png/testing/0/1197.png',
       b'./mnist_png/testing/0/1203.png',
       b'./mnist_png/testing/0/1218.png',
       b'./mnist_png/testing/0/1220.png',
       b'./mnist_png/testing/0/1223.png',
       b'./mnist_png/testing/0/1231.png',
       b'./mnist_png/testing/0/1246.png',
       b'./mnist_png/testing/0/126.png',
   